#**Transformers con Hugging Face**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00


**Primer Modelo**

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
import torch
import torch.nn.functional as F


Sentiment Analysis

In [ ]:
classifier =  pipeline("sentiment-analysis")
res = classifier("Me gusta esta pelicula")



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
print(res)

[{'label': 'POSITIVE', 'score': 0.975827693939209}]


Ver otros pipelines: https://huggingface.co/transformers/v3.2.0/main_classes/pipelines.html#

**Segundo Modelo**

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" #distilbert-base-uncased-finetuned-sst-2-english"
classifier =  pipeline("sentiment-analysis", model=model_name)

In [ ]:
results = classifier(["we are happy","we are sad", "I feel hungry"])
for result in results:
	print(result)

{'label': '5 stars', 'score': 0.5690038204193115}
{'label': '1 star', 'score': 0.3714503049850464}
{'label': '2 stars', 'score': 0.3204993009567261}


EN español?

buscar un modelo aqui: https://huggingface.co/models


In [ ]:
model_name = "finiteautomata/beto-sentiment-analysis"
clasificador =  pipeline("sentiment-analysis", model=model_name)
results = clasificador(["No fue buena la atención","La atención fue más o menos", "cierren mi cuenta"])
for result in results:
	print(result)

{'label': 'NEG', 'score': 0.9947535991668701}
{'label': 'NEU', 'score': 0.9979003667831421}
{'label': 'NEG', 'score': 0.7338854670524597}


**Tercer Modelo**

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier =  pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [ ]:
results = classifier(["we are happy","we are sad"])

for result in results:
	print(result)

{'label': 'POSITIVE', 'score': 0.9998812675476074}
{'label': 'NEGATIVE', 'score': 0.9977351427078247}


Tokenizer

In [ ]:
tokens = tokenizer.tokenize("we are very upset")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("we are very upset")

print(f' Tokens: {tokens}')
print(f' Token IDs: {token_ids}')
print(f' Input IDs: {input_ids}')


 Tokens: ['we', 'are', 'very', 'upset']
 Token IDs: [2057, 2024, 2200, 6314]
 Input IDs: {'input_ids': [101, 2057, 2024, 2200, 6314, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}


**Cuarto Modelo**

In [ ]:
X_train = ["we are happy","we are sad"]
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt") # pt means PyTorch
print(batch)


{'input_ids': tensor([[ 101, 2057, 2024, 3407,  102],
        [ 101, 2057, 2024, 6517,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


In [ ]:
with torch.no_grad():
	outputs = model(**batch, labels=torch.tensor([1,0]))
	print(outputs)
	predictions = F.softmax(outputs.logits, dim=1)
	print(predictions)
	labels = torch.argmax(predictions, dim=1)
	print(labels)
	labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
	print(labels)


SequenceClassifierOutput(loss=tensor(0.0012), logits=tensor([[-4.3440,  4.6950],
        [ 3.2917, -2.7962]]), hidden_states=None, attentions=None)
tensor([[1.1868e-04, 9.9988e-01],
        [9.9774e-01, 2.2648e-03]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


**Guardando un modelo**

In [ ]:
#saving
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)
#carga
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory) #


**Quinto Modelo: Español**

Sentiment analysis

In [ ]:
model_name = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)



In [ ]:
X_train_esp = ["No me gusta el banco", "La atención es muy buena", "mas o menos","Bastante mala la atención"]
batch = tokenizer(X_train_esp, padding=True, truncation=True, max_length=512, return_tensors="pt") # pt means PyTorch


In [ ]:
with torch.no_grad():
	outputs =  model(**batch)
	label_ids =  torch.argmax(outputs.logits, dim=1)
	print(label_ids)
	labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
	print(labels)



tensor([0, 2, 1, 0])
['NEG', 'POS', 'NEU', 'NEG']


#**Transfer Learning en NLP con Transformers**

Book: Natural Language with Transformers - Lewis Tunstall

![](https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_transfer-learning.png?raw=1)

**Tareas que pueden ser resueltas con Transformers**

**Clasificación de Texto**

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


**NER**

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


In [ ]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


**Text Summarization**

In [ ]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I


**Machine Translation**

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00


In [ ]:
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

ValueError: ignored

**Generación de Texto**

In [ ]:
#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [ ]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. You have requested a new Transformers 2 action figure and the Optimus Prime action figure I received on June 17 was not a part of this purchase. After receiving your gift, I have decided to call and request a new Transformers 2 action figure, or even a new Hasbro action figure. I can confirm that in order to resolve these issues,


**Principales Modelos Transformers**

![](https://raw.githubusercontent.com/nlp-with-transformers/notebooks/884293a3ab50071aa2afd1329ecf4a24f0793333//images/chapter03_transformers-compact.png)

**Computer Vision**

![](https://raw.githubusercontent.com/nlp-with-transformers/notebooks/884293a3ab50071aa2afd1329ecf4a24f0793333//images/chapter11_vit-architecture.png)